this is a notebook for speech siamese. 
the goal is to add siamese network after the speech command network to make a one-shot speech command model. with this model, take two piece of audio as input, the model will tell if it is the same speech command or not. 
if the accuracy is good enough, we make take it input product for voice trigger or voice command which are useful for all kind of product. 

the trick may be if siamese can make one shot accure enough. 

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import hashlib
import math, time, datetime
import os.path
import random
import re
import sys
import tarfile

import matplotlib.pyplot as plt
import numpy as np
import librosa as rosa
import librosa.display
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Lambda, BatchNormalization, Activation, LSTM, GRU
#from tensorflow.contrib.framework.python.ops import audio_ops as contrib_audio
#from tensorflow.python.ops import io_ops
#from tensorflow.python.platform import gfile
#from tensorflow.python.util import compat


# binary data is n * 32 * 128
default_mfcc_length = 32
default_number_of_mfcc = 128

default_epochs=10
default_batch_size=32

# data have been writed to binary file, no need to input words, just for reference
#reference start
default_wanted_words=["one", "two", "bed", "backward", "bird", "cat", "dog", "eight", "five", "follow", "forward", "four", "go", "happy", "house", "learn", "left", "marvin", "nine", "no", "off", "right", "seven", "sheila", "stop", "three", "tree", "visual", "wow", "zero","up"]
default_train_words=["one", "two", "cat", "dog", "bed", "backward", "eight","forward", "four", "go", "happy", "house", "learn", "left", "marvin", "nine", "no", "off", "right", "seven", "sheila", "stop", "three", "tree", "visual", "wow", "zero","up"]
default_test_words=["five", "follow", "bird"]
#reference end

mfcc_data_train_mfcc1 = "/home/zhangjun/tensorflow/speech_siamese_zj/binary_data/train_scaler/mfcc1_train_scaler_(50000, 32, 128)_2018-11-08.npy"
mfcc_data_train_mfcc2 = "/home/zhangjun/tensorflow/speech_siamese_zj/binary_data/train_scaler/mfcc2_train_scaler_(50000, 32, 128)_2018-11-08.npy"
mfcc_data_train_label = "/home/zhangjun/tensorflow/speech_siamese_zj/binary_data/train_scaler/label_train_scaler_(50000,)_2018-11-08.npy"

mfcc_data_val_mfcc1 = "/home/zhangjun/tensorflow/speech_siamese_zj/binary_data/validation_scaler/mfcc1_validation_scaler_(100, 32, 128)_2018-11-08.npy"
mfcc_data_val_mfcc2 = "/home/zhangjun/tensorflow/speech_siamese_zj/binary_data/validation_scaler/mfcc2_validation_scaler_(100, 32, 128)_2018-11-08.npy"
mfcc_data_val_label = "/home/zhangjun/tensorflow/speech_siamese_zj/binary_data/validation_scaler/label_validation_scaler_(100,)_2018-11-08.npy"

mfcc_data_test_mfcc1 = "/home/zhangjun/tensorflow/speech_siamese_zj/binary_data/test_scaler/mfcc1_test_scaler_(100, 32, 128)_2018-11-08.npy"
mfcc_data_test_mfcc2 = "/home/zhangjun/tensorflow/speech_siamese_zj/binary_data/test_scaler/mfcc2_test_scaler_(100, 32, 128)_2018-11-08.npy"
mfcc_data_test_label = "/home/zhangjun/tensorflow/speech_siamese_zj/binary_data/test_scaler/same_test_scaler_(100,)_2018-11-08.npy"


default_model_path="/home/zhangjun/tensorflow/speech_siamese_zj/trained"



# LSTM Network

create a keras lstm network, take mfcc vector as input.

the speech command mfcc input shape is (?, mfcc_number, hop_number, 1)




In [2]:
def read_mfcc_data(file_name):
    f = open(file_name, "rb")
    t = np.load(f)
    print("shape:", t.shape)
    return t


In [3]:
def create_lstm_model(local_input_shape, is_training=True):
    model = Sequential()
    model.add(GRU(256, return_sequences=False, stateful=False, input_shape=local_input_shape))
    #model.add(GRU(256, return_sequences=True, stateful=False))
    #model.add(GRU(256, stateful=False))

    model.add(Dense(256))
    model.add(BatchNormalization())
    model.add(Activation("sigmoid")) 
    #if (is_training):
    #    model.add(Dropout(0.5))
    #model.add(Dense(labels_count, activation="softmax"))
    
    return model

def create_siamese_model(input_shape, siamese_mode = 'concat'):
    right_input = Input(input_shape)
    left_input = Input(input_shape)
    keras_model = create_lstm_model(input_shape)
    
    right_encoder = keras_model(right_input)
    left_encoder = keras_model(left_input)
    if (siamese_mode == 'minus'):
        concatenated_layer = Lambda(lambda x: x[0]-x[1], output_shape=lambda x: x[0])([right_encoder, left_encoder])
    elif (siamese_mode == 'abs'):
        concatenated_layer = Lambda(lambda x: tf.abs(x[0]-x[1]), output_shape=lambda x: x[0])([right_encoder, left_encoder])
    #elif (siamese_mode == "eu"):
    #    concatenated_layer = Lambda(lambda x: tf.sqrt(tf.reduce_sum(tf.square(x[0]-x[1]), 2)), output_shape=lambda x: x[0])([right_encoder, left_encoder])
    else:
        raise ValueError("unknown siamese_mode")
        
    output_layer = Dense(1, activation='sigmoid')(concatenated_layer)
    siamese_model = Model([right_input, left_input], output_layer)
    return siamese_model
    
def siamese_train(local_siamese_mode='abs'):
    siamese_model = create_siamese_model((default_mfcc_length, default_number_of_mfcc), siamese_mode=local_siamese_mode)

    siamese_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    x1_train = read_mfcc_data(mfcc_data_train_mfcc1)
    x2_train = read_mfcc_data(mfcc_data_train_mfcc2) 
    y_train = read_mfcc_data(mfcc_data_train_label)
    
    
    siamese_model.fit([x1_train, x2_train], y_train, epochs=default_epochs, batch_size=default_batch_size)
    
    
    x1_test = read_mfcc_data(mfcc_data_val_mfcc1)
    x2_test = read_mfcc_data(mfcc_data_val_mfcc2)
    y_test = read_mfcc_data(mfcc_data_val_label) 
    
    loss, accuracy = siamese_model.evaluate([x1_test, x2_test], y_test)    
    
    siamese_model.save(default_model_path+"/speech_siamese.h5")

    print(loss)
    return accuracy

def siamese_test():
    siamese_model = keras.models.load_model(default_model_path+"/speech_siamese.h5")    
    
    x1_test = read_mfcc_data(mfcc_data_test_mfcc1) 
    x2_test = read_mfcc_data(mfcc_data_test_mfcc2) 
    y_test = read_mfcc_data(mfcc_data_test_label)
    
    loss, accuracy = siamese_model.test_on_batch(x=[x1_test, x2_test], y=y_test)
    print(loss)
    return accuracy

Siamese Network

## main 

In [ ]:
#wav_mfcc = load_wav_mfcc("/Users/hermitwang/Downloads/speech_dataset/backward/0a2b400e_nohash_0.wav")
#print(wav_mfcc.shape) 
score=siamese_train(local_siamese_mode='abs')
print(score)
score=siamese_test()

print(score)

shape: (50000, 32, 128)
shape: (50000, 32, 128)
shape: (50000,)
Epoch 1/10
50000/50000 [==============================] - 159s 3ms/step - loss: 0.6873 - acc: 0.5463
Epoch 2/10
20896/50000 [===========>..................] - ETA: 1:30 - loss: 0.6695 - acc: 0.5847

# the fact

Epoch 1/10
10000/10000 [==============================] - 108s 11ms/step - loss: 0.6679 - acc: 0.5937
Epoch 2/10
10000/10000 [==============================] - 99s 10ms/step - loss: 0.5817 - acc: 0.6971
Epoch 3/10
10000/10000 [==============================] - 99s 10ms/step - loss: 0.5324 - acc: 0.7306
Epoch 4/10
10000/10000 [==============================] - 96s 10ms/step - loss: 0.4799 - acc: 0.7684
Epoch 5/10
10000/10000 [==============================] - 97s 10ms/step - loss: 0.4271 - acc: 0.8066
Epoch 6/10
10000/10000 [==============================] - 96s 10ms/step - loss: 0.3796 - acc: 0.8310
Epoch 7/10
10000/10000 [==============================] - 97s 10ms/step - loss: 0.3459 - acc: 0.8511
Epoch 8/10
10000/10000 [==============================] - 97s 10ms/step - loss: 0.3103 - acc: 0.8694
Epoch 9/10
10000/10000 [==============================] - 96s 10ms/step - loss: 0.2784 - acc: 0.8844
Epoch 10/10
10000/10000 [==============================] - 96s 10ms/step - loss: 0.2480 - acc: 0.8979
100/100 [==============================] - 1s 11ms/step
0.5532743597030639
0.79
1.0078126
0.63

GRU*256 -> Dense 1024
Epoch 1/10
10000/10000 [==============================] - 108s 11ms/step - loss: 0.6706 - acc: 0.5843
Epoch 2/10
10000/10000 [==============================] - 99s 10ms/step - loss: 0.5864 - acc: 0.6882
Epoch 3/10
10000/10000 [==============================] - 97s 10ms/step - loss: 0.5325 - acc: 0.7314
Epoch 4/10
10000/10000 [==============================] - 97s 10ms/step - loss: 0.4816 - acc: 0.7734
Epoch 5/10
10000/10000 [==============================] - 97s 10ms/step - loss: 0.4413 - acc: 0.7942
Epoch 6/10
10000/10000 [==============================] - 97s 10ms/step - loss: 0.3941 - acc: 0.8206
Epoch 7/10
10000/10000 [==============================] - 97s 10ms/step - loss: 0.3655 - acc: 0.8393
Epoch 8/10
10000/10000 [==============================] - 98s 10ms/step - loss: 0.3162 - acc: 0.8612
Epoch 9/10
10000/10000 [==============================] - 97s 10ms/step - loss: 0.2839 - acc: 0.8804
Epoch 10/10
10000/10000 [==============================] - 98s 10ms/step - loss: 0.2514 - acc: 0.8980
100/100 [==============================] - 1s 13ms/step
0.6746552109718322
0.71
0.7804642
0.62


3*GRU * 256 -> Dense 1024
Epoch 1/10
10000/10000 [==============================] - 380s 38ms/step - loss: 0.6932 - acc: 0.5577
Epoch 2/10
10000/10000 [==============================] - 364s 36ms/step - loss: 0.6512 - acc: 0.6203
Epoch 3/10
10000/10000 [==============================] - 363s 36ms/step - loss: 0.6079 - acc: 0.6744
Epoch 4/10
10000/10000 [==============================] - 363s 36ms/step - loss: 0.5538 - acc: 0.7181
Epoch 5/10
10000/10000 [==============================] - 365s 36ms/step - loss: 0.5094 - acc: 0.7537
Epoch 6/10
10000/10000 [==============================] - 364s 36ms/step - loss: 0.4635 - acc: 0.7846
Epoch 7/10
10000/10000 [==============================] - 362s 36ms/step - loss: 0.4153 - acc: 0.8133
Epoch 8/10
10000/10000 [==============================] - 366s 37ms/step - loss: 0.3677 - acc: 0.8407
Epoch 9/10
10000/10000 [==============================] - 363s 36ms/step - loss: 0.3252 - acc: 0.8616
Epoch 10/10
10000/10000 [==============================] - 366s 37ms/step - loss: 0.2846 - acc: 0.8786
100/100 [==============================] - 1s 14ms/step
0.6436352944374084
0.72
0.812591
0.67

GRU*256-> Dense 256
Epoch 1/10
10000/10000 [==============================] - 103s 10ms/step - loss: 0.6787 - acc: 0.5622
Epoch 2/10
10000/10000 [==============================] - 95s 10ms/step - loss: 0.6169 - acc: 0.6618
Epoch 3/10
10000/10000 [==============================] - 96s 10ms/step - loss: 0.5689 - acc: 0.7099
Epoch 4/10
10000/10000 [==============================] - 94s 9ms/step - loss: 0.5268 - acc: 0.7364
Epoch 5/10
10000/10000 [==============================] - 93s 9ms/step - loss: 0.4821 - acc: 0.7714
Epoch 6/10
10000/10000 [==============================] - 94s 9ms/step - loss: 0.4511 - acc: 0.7908
Epoch 7/10
10000/10000 [==============================] - 94s 9ms/step - loss: 0.4146 - acc: 0.8090
Epoch 8/10
10000/10000 [==============================] - 93s 9ms/step - loss: 0.3784 - acc: 0.8345
Epoch 9/10
10000/10000 [==============================] - 94s 9ms/step - loss: 0.3437 - acc: 0.8474
Epoch 10/10
10000/10000 [==============================] - 94s 9ms/step - loss: 0.3018 - acc: 0.8764
100/100 [==============================] - 2s 22ms/step
0.4378182792663574
0.76
0.58927524
0.73

Epoch 1/10
10000/10000 [==============================] - 107s 11ms/step - loss: 0.6708 - acc: 0.5770
Epoch 2/10
10000/10000 [==============================] - 97s 10ms/step - loss: 0.6115 - acc: 0.6622
Epoch 3/10
10000/10000 [==============================] - 95s 10ms/step - loss: 0.5634 - acc: 0.7096
Epoch 4/10
10000/10000 [==============================] - 95s 9ms/step - loss: 0.5310 - acc: 0.7351
Epoch 5/10
10000/10000 [==============================] - 95s 10ms/step - loss: 0.4971 - acc: 0.7621
Epoch 6/10
10000/10000 [==============================] - 99s 10ms/step - loss: 0.4523 - acc: 0.7915
Epoch 7/10
10000/10000 [==============================] - 97s 10ms/step - loss: 0.4247 - acc: 0.8069
Epoch 8/10
10000/10000 [==============================] - 97s 10ms/step - loss: 0.3903 - acc: 0.8284
Epoch 9/10
10000/10000 [==============================] - 97s 10ms/step - loss: 0.3569 - acc: 0.8445
Epoch 10/10
10000/10000 [==============================] - 98s 10ms/step - loss: 0.3255 - acc: 0.8616
100/100 [==============================] - 3s 26ms/step
0.5993771600723267
0.71
0.8117188
0.67


